<style>
/* Reveal container full viewport */
.reveal {
  width: 95vw !important; /* 95% of view width */
  height: 50vw !important;   /* about 16:9 aspect ratio height (width × 9/16) */
  max-width: 95vw !important;
  max-height: 50vw !important;
  margin: auto;
  font-size: 110%; /* global font size */
}

/* Slides filling container */
.reveal .slides {
  width: 100% !important;
  height: 100% !important;
  max-width: 100% !important;
  max-height: 100% !important;
  margin: auto;
  overflow-wrap: break-word;
}

/* Prevent content from overflowing horizontally */
.reveal section {
  overflow-x: auto;
  word-wrap: break-word;
  box-sizing: border-box;
  padding: 1em;
}

/* Tables scroll horizontally if too wide */
.reveal section table {
  display: block;
  overflow-x: auto;
  white-space: nowrap;
  max-width: 100%;
  margin: 0 auto;
  font-size: 90%;
  border-collapse: collapse;
}

/* Smaller font and compact padding inside table cells */
.reveal section table td,
.reveal section table th {
  font-size: 90%;
  padding: 0.3em 0.5em;
  white-space: nowrap;
}
</style>

# Second Hands-On Session
--- 
Tracking many particles - Twiss parameters

---

## 🐍 Python corner

We'll be using:
- `numpy` as `np`
- `matplotlib.pyplot` as `plt`
- functions `D`, `Q`, `transportParticles`, `getEquivalentElement` and `transportSigmas`

In [1]:
from tracking_library import *

numpy is installed, version: 2.3.1
scipy is installed, version: 1.15.3
matplotlib is installed, version: 3.10.0
-> Setup is OK! Have fun!


## 📝 Exercise 2.1 (guided): Generating an ensemble of particles <a id="exercise-2.1-guided"></a>

Define an ensemble of 100 particles with arbitrary first order (e.g. $<x_0> = 0.2$ mm, $<x'_0> = 1$ mrad) and second order (e.g. $\sigma_{x_0} = 1.5$ mm and $\sigma_{x'_0} = 0.5$ mrad) momenta.
Verify that the angular divergence of the beam is the one set: 

- What do you observe?
- What happens if you increase or decrease the number of particles?

> 👀 **HINT**: Remember that you can create a Normal distributed 2xN 2D array of numbers as `np.random.randn(2, N)`. One can then "stretch" and "offset" (and eventually "rotate", but this is not requested here) this distribution to obtain any desired 2D normal distribution in phase space.

In [2]:
# simple solution
N_particles = 100
beam = np.array([ np.random.randn(1, N_particles), 
                  np.random.randn(1, N_particles) ])

In [3]:
beam

array([[[ 0.47838492, -0.22214722,  0.97130241, -0.46691753,
          1.16093174, -0.29668437,  0.58047458,  0.21152941,
         -0.93003438,  0.45532431,  0.1918376 ,  1.16963089,
          0.71321022,  0.85639502,  0.57007002, -1.50502519,
         -1.02423131, -0.10478933, -1.03180741, -0.96030343,
         -2.16206389,  1.09969421, -1.69660878,  0.3743274 ,
          1.99147935,  0.44938877,  0.0576456 , -0.01610862,
          1.21039062, -1.16505268,  0.43461637, -0.41618585,
         -0.32970073,  2.09240082, -0.22332468,  0.73077574,
         -0.90584645, -0.43766516, -0.09581515, -2.43094808,
         -0.68341049,  0.38987777, -1.58743217, -0.93271421,
         -0.91181719,  0.18109887, -0.74831093,  1.73431157,
          0.24748301,  1.25728915,  0.42211699,  1.78053903,
          1.47273761, -0.21540128,  0.7834155 ,  1.76434948,
          0.47938842, -0.2260167 , -1.06337884,  0.4836769 ,
         -0.8373384 ,  0.45171768, -2.03509073, -0.6265273 ,
          0.43206384,  1

In [17]:
# or equivalently:
N_particles = 50000
beam = np.random.randn(2, N_particles)

In [18]:
x0 = 0.2
xp0 = 1
sigx = 1.5
sigxp = 0.5
# add a scaling (sigx*) and an offset (+x0) to the x (0) coordinate:
beam[0, :] = sigx * beam[0, :] + x0
# add a scaling (sigxp*) and an offset (+xp0) to the x' (1) coordinate:
beam[1, :] = sigxp * beam[1, :] + xp0

In [19]:
# compute the means and std of the particle coordinates
means = np.mean(beam, axis=1)
stds  = np.std(beam, axis=1)

In [20]:
# print the actual means and stds over all particle coordinates
print(f"Beam mean  x [mm]: {means[0]}")
print(f"Beam mean  x' [mrad]: {means[1]}")
print(f"Beam rms size [mm]: {stds[0]}")
print(f"Beam rms divergence [mrad]: {stds[1]}")

Beam mean  x [mm]: 0.20215524538003607
Beam mean  x' [mrad]: 0.9997408437052792
Beam rms size [mm]: 1.492602567236973
Beam rms divergence [mrad]: 0.5003655616614343


❓**QUESTION: Why didn't we find back the input parameters?**

## 📝 Exercise 2.2: Tracking an ensemble of particles in a drift <a id="exercise-2.2"></a>

1. Transport the beam distribution of [Exercise 2.1](#exercise-2.1-guided) in a drift of length 1 m. Visualise the initial and final distribution. **What do you observe?**

   > 👀 **HINT 1**: the output of `transportParticles` contains the coordinates of all particles at all locations. To obtain all particle coordinates at the end of the beamline, you can simply do:
   >
   > ```
   > tracked = transportParticles(initial_coordinates, beamline)
   > final_coordinates = tracked['coords'][-1] #where "-1" refers to the last element of the beamline
   > ```

   > 👀 **HINT 2**: One can use `plt.plot` or `plt.scatter` to plot the 2D distribution, e.g.:
   > ```
   > plt.scatter(x, xp, marker='.')
   > ```

2. (Optional) Test of linearity. Scale the input vector by 17 times the month of your birthday (e.g. 85 if you are born in May) and verify that the output vector from the matrix multiplication has changed by the same factor.
    
    > 👀 **HINT**: Be careful with machine precision!


In [ ]:
# code here your solution...

## 📝 Exercise 2.3: Tracking an ensemble of particles along a beamline (mean position & beam size) <a id="exercise-2.3"></a>

Build a beamline made of several drift and quadrupoles as desired (e.g. `D(0.5)+Q(1)+D(2)+Q(-1)+D(1)+Q(2)+....`).

Build a beam made of several particles (e.g. 1000) again with arbitrary first order (e.g. $<x_0> = 0.2$ mm, $<x'_0> = 1$ mrad) and second order (e.g. $\sigma_{x_0} = 1.5$ mm and $\sigma_{x'_0} = 0.5$ mrad) momenta, as done in previous [Exercise 2.2](#exercise-2.2).

1. Plot the individual particle trajectories along the beamline

2. Compute and plot the average beam position, the beam size ($\sigma_x$) and divergence ($\sigma_{x'}$) along the beam line.

> 👀 **HINT 1 (Python)**: Remember that in the output of our `transportParticles` function the key `'x'` contains a 2D array with N rows (the index of the position along the beam line) and M columns (the index of a given particle). 
You can compute the standard deviation of **each raw** of a NxM 2D array as `np.std(N_times_M_array,1)`. 

> 👀 **HINT 2 (Python)**: After having plotted $x$ trajectory on a matplotlib plot, one can create a **second vertical axis** that shares the same horizontal axis with `plt.twinx()`. This can be convenient to see, for example, both $\sigma_x$ and $\sigma_{x'}$ on the same plot.


In [ ]:
# code here your solution...

## ⚛️ Physics focus: Sigma matrices

One can easily demonstrate (see [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/)) that the same matrix ($M$) used for tracking the coordinates ($(x_i, x'_i)$) of each single particle ($i$) from an initial point ($X_0$) to a final point ($X_s$) in a beamline:

\begin{equation}
X_s =  M\, X_0
\end{equation}

can also be used to track the **average trajectory** ($\langle X \rangle$) as well as the **covariance or sigma matrix** of the given particle coordinates distribution:

\begin{equation}
\langle X_s \rangle = 
\left[
\begin{array}{c}
\langle x_i \rangle\\
\langle x'_i \rangle
\end{array}
\right]_s 
= M\, \langle X_0 \rangle
\end{equation}

\begin{equation}
\Sigma_s = \left[
\begin{array}{c}
\sigma_{xx}\quad \sigma_{xx'}\\
\sigma_{x'x}\quad \sigma_{x'x'}
\end{array}
\right]_s
= M\, \Sigma_0\, M^T\, .
\end{equation}

We can therefore track the **average trajectory** and **covariance** of a beam simply starting from its initial average coordinates and covariance matrix in phase space.

The "tracking" of an initial **covariance** matrix along a given beamline is provided by the function `transportSigmas()` function from our toolbox:

In [ ]:
from tracking_library import transportSigmas

# let's see if there is some help information:
help(transportSigmas)

## 📝 Exercise 2.4: Tracking only the average beam position

Show that **the average position of a beam along a beam line** (e.g. the beamline and beam you defined in [Exercise 2.3](#exercise-2.3)) is the same as **the trajectory of single particle that starts in the center of the initial particle distribution**.

In [ ]:
# code here your solution...

## 📝 Excercise 2.5: Tracking only the beam size

For the same system as before, track only the sigma matrix and compare with the rms beam size computed from tracking all particles (Exercise 2.3).

> 🔹 **NOTE**: Is this valid for any number of initially tracked particles? How does the result change if one uses the **input** covariance matrix used to generate the particle distribution rather then the **actual** covariance matrix of the generated distribution?

> 👀 **HINT 1**: Remember that the element $\sigma_{xx}$ of the covariance matrix is linked to the rms beam size ($\sigma_x$) as $\sigma_x = \sqrt{\sigma_{xx}}$.

> 👀 **HINT 2**: The covariance matrix of a 2xN array can be computed using `numpy` as `np.cov(2_times_N_array, bias=True)`.

In [ ]:
# code here your solution...

## ⚛️ Physics focus: Introduction of Twiss values and emittance

Recall that:

\begin{equation}
\Sigma_{s} = M\, \Sigma_{0}\, M^T\, .
\end{equation}

where $M$ is a real **symplectic** transformation, and its determinant is $\det(M) = +1$. Therefore:

\begin{equation}
\det(\Sigma_s) = \det( M\, \Sigma_0\, M^T ) = \det(M) \det(\Sigma_0) \det(M^T) = \det(\Sigma_0)
\end{equation}

the determinant of the sigma/covariance matrix is **preserved** along a beamline. The sigma/covariance matrix of any particle distribution can be parameterized as:

\begin{equation}
\Sigma = 
    \left[
    \begin{array}{cc}
    \sigma_{xx}  & \sigma_{xx'}\\
    \sigma_{x'x} & \sigma_{x'x'}
    \end{array}
    \right] =
    \epsilon
    \left[ 
    \begin{array}{cc}
        \beta   & -\alpha\\
        -\alpha & \gamma
    \end{array}
    \right] 
\end{equation}

where $\beta$, $\alpha$, $\gamma$ and $\epsilon$ are parameters such that $\epsilon = \sqrt{\det(\Sigma)}$ (the beam **statistical emittance**) and $\beta \gamma - \alpha^2 = 1$. The *Twiss* parmeters $\beta, \gamma, \alpha$ define the **normalised** shape/orientation of the beam distribution in phase-space!

**This seems to be an arbitrary choice!** but it will acquire more special meaning later.

## 📝 Exercise 2.6: Equivalent matrix symplecticity
Verify that the equivalent transport matrix of any beamline, e.g. the one you used in [Exercise 2.3](#exercise-2.3), has determinant equal to 1 (within machine precision).

> 👀 **HINT (Python)**: you can use `np.linalg.det(matrix)` to compute the determinant of a matrix `matrix`

In [ ]:
# code here your solution...

## 📝 Exercise 2.7: Evolution of Twiss parameters in a beamline

Consider again a beamline, e.g. the one you used in [Exercise 2.3](#exercise-2.3), and create a valid sigma matrix with:
- $\beta$ = 3 [m]
- $\gamma$ = 0.5 [1/m]
- $\epsilon$ = 5 [$\mu$ m]

Then, propagate the $\sigma$ matrix through the beam line and verify that the emittance $\epsilon$ of the sigma matrix after every element is indeed constant and equal to its initial value.

**Optional:** compute and plot the **beta** (function) all along the beamline, i.e. $\sigma_{11}/\epsilon$

> 👀 **HINT (Python)**: in the output of our `transportSigmas()` function we keep all sigma matrixes. The determinant of all matrices can be computed in one go as `np.linalg.det(transported_sigmas['sigmas'])`.

In [ ]:
# code here your solution...

## Well done !!

Now you understand how to transport an ensamble of particles or its **covariance** matrix along a beamline... But how to design a "nice" beamline?

=> **Continue your learning with the following [notebook](./03_Periodic_Systems.ipynb)**...
